In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
class MyClass():
  def __init__(self):
    self.var1 = 1

my_obj = MyClass()

In [3]:
my_obj.__dict__

{'var1': 1}

In [4]:
my_obj.var2 = 2
vars(my_obj)

{'var1': 1, 'var2': 2}

In [5]:
vars(my_obj)['var3'] = 3
vars(my_obj)

{'var1': 1, 'var2': 2, 'var3': 3}

In [6]:
for i in range(4,10):
  vars(my_obj)[f'var{i}'] = 0

vars(my_obj)

{'var1': 1,
 'var2': 2,
 'var3': 3,
 'var4': 0,
 'var5': 0,
 'var6': 0,
 'var7': 0,
 'var8': 0,
 'var9': 0}

### Create a generic VGG block (TODO)

In [7]:
class Block(tf.keras.Model):
  def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
    super(Block, self).__init__()
    self.filters = filters
    self.kernel_size = kernel_size
    self.repetitions = repetitions

    for i in range(self.repetitions):
      vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(filters=self.filters,
                                                         kernel_size=self.kernel_size,
                                                         activation='relu',
                                                         padding='same')

    self.max_pool = tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=strides)

  def call(self, inputs):
    conv2D_0 = self.conv2D_0
    x = conv2D_0(inputs)

    for i in range(1, self.repetitions):
      conv2D_i = vars(self)[f'conv2D_{i}']
      x = conv2D_i(x)

    max_pool = self.max_pool(x)

    return max_pool

### Create the Custom VGG network (TODO)

In [8]:
class MyVGG(tf.keras.Model):

  def __init__(self, num_classes):
    super(MyVGG, self).__init__()

    self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
    self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
    self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
    self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
    self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

    self.flatten = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(256, activation='relu')
    self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    x = self.block_a(inputs)
    x = self.block_b(x)
    x = self.block_c(x)
    x = self.block_d(x)
    x = self.block_e(x)
    x = self.flatten(x)
    x = self.fc(x)
    x = self.classifier(x)
    return x

In [ ]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')
vgg = MyVGG(num_classes=2)

vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


def preprocess(features):
  image = tf.image.resize(features['image'], (224, 224))
  return tf.cast(image, tf.float32) / 255. , features['label']

dataset = dataset.map(preprocess).batch(32)

vgg.fit(dataset, epochs=10)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling data/cats_vs_dogs/incomplete.DZ177D_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%|          | 0/23262 …

Dataset cats_vs_dogs downloaded and prepared to data/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
Epoch 1/10
  2/727 [..............................] - ETA: 3:52:45 - loss: 0.6895 - accuracy: 0.5625